In [1]:
!pyspark --version

22/02/27 10:58:51 WARN Utils: Your hostname, thomas-comp resolves to a loopback address: 127.0.1.1; using 192.168.0.160 instead (on interface enp34s0)
22/02/27 10:58:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.2.1
      /_/
                        
Using Scala version 2.12.15, OpenJDK 64-Bit Server VM, 11.0.13
Branch HEAD
Compiled by user hgao on 2022-01-20T19:26:14Z
Revision 4f25b3f71238a00508a356591553f2dfa89f8290
Url https://github.com/apache/spark
Type --help for more information.


In [43]:
import pyspark
from pyspark.sql import SparkSession

In [44]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("test") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

In [7]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv

--2022-02-27 11:11:35--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv
Распознаётся nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.217.87.100
Подключение к nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.87.100|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа... 200 OK
Длина: 733822658 (700M) [text/csv]
Сохранение в каталог: ««fhvhv_tripdata_2021-02.csv.1»».

fhvhv_tripdata_2021 100%[===================>] 699,83M  1,80MB/s    за 4m 11s  

2022-02-27 11:15:47 (2,79 MB/s) - «fhvhv_tripdata_2021-02.csv.1» сохранён [733822658/733822658]



In [6]:
!wc -l fhvhv_tripdata_2021-02.csv

11613943 fhvhv_tripdata_2021-02.csv


In [45]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-02.csv')

In [46]:
import pandas as pd

In [47]:
df_pandas = pd.read_csv('head.csv')

In [48]:
spark.createDataFrame(df_pandas).schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,LongType,true),StructField(DOLocationID,LongType,true),StructField(SR_Flag,DoubleType,true)))

In [49]:
from pyspark.sql import types

In [50]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [51]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-02.csv')

In [52]:
df = df.repartition(24)

In [53]:
df.write.parquet('fhvhv/2021/01/', mode='overwrite')

22/02/27 13:42:57 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
22/02/27 13:42:57 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 84,44% for 9 writers
22/02/27 13:42:57 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 76,00% for 10 writers
22/02/27 13:42:57 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 69,09% for 11 writers
22/02/27 13:42:57 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 63,33% for 12 writers
22/02/27 13:42:58 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 69,09% for 11 writers
22/02/27 13:42:58 WARN MemoryManager: Total allocation exceeds 95,

In [54]:
from pyspark.sql import functions as F

In [55]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .select('pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-----------+------------+------------+------------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-----------+------------+------------+------------+
| 2021-02-01|  2021-02-01|         106|         106|
| 2021-02-02|  2021-02-02|          37|          25|
| 2021-02-01|  2021-02-01|          62|          61|
| 2021-02-03|  2021-02-03|          18|         147|
| 2021-02-01|  2021-02-01|          74|          74|
| 2021-02-01|  2021-02-01|          76|         117|
| 2021-02-01|  2021-02-01|         191|          98|
| 2021-02-03|  2021-02-03|          48|         265|
| 2021-02-03|  2021-02-03|          17|         217|
| 2021-02-03|  2021-02-03|         236|         170|
| 2021-02-03|  2021-02-04|         162|         145|
| 2021-02-02|  2021-02-02|         243|         162|
| 2021-02-03|  2021-02-03|         138|          75|
| 2021-02-04|  2021-02-04|          67|         228|
| 2021-02-02|  2021-02-02|         130|         180|
| 2021-02-02|  2021-02-02|          47|       

In [81]:
from pyspark.sql import functions as F

In [89]:
new_df = df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('long_of_trip', (F.col("dropoff_datetime").cast("long") - F.col("pickup_datetime").cast("long"))/60) \
    .select('pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID', 'long_of_trip')

In [91]:
new_df.orderBy(F.col("long_of_trip").desc()).show()

+-----------+------------+------------+------------+------------------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|      long_of_trip|
+-----------+------------+------------+------------+------------------+
| 2021-02-11|  2021-02-12|         247|          41|            1259.0|
| 2021-02-17|  2021-02-18|         242|         254| 953.6833333333333|
| 2021-02-20|  2021-02-21|         188|          55| 733.9833333333333|
| 2021-02-03|  2021-02-04|          51|         147|            677.55|
| 2021-02-19|  2021-02-20|         210|         149| 626.2833333333333|
| 2021-02-25|  2021-02-26|         174|         126|             583.5|
| 2021-02-20|  2021-02-20|         242|          31|             580.1|
| 2021-02-18|  2021-02-19|         196|         197| 576.8666666666667|
| 2021-02-18|  2021-02-18|          89|         265| 575.9166666666666|
| 2021-02-10|  2021-02-11|         254|         259| 569.4833333333333|
| 2021-02-10|  2021-02-10|          61|         265| 541.2666666

In [92]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02867|2021-02-02 22:50:03|2021-02-02 23:12:18|         134|         179|   null|
|           HV0003|              B02872|2021-02-03 07:44:08|2021-02-03 08:00:06|          48|         140|   null|
|           HV0003|              B02871|2021-02-02 18:45:47|2021-02-02 19:04:51|          39|          35|   null|
|           HV0003|              B02871|2021-02-01 19:36:44|2021-02-01 19:45:31|         226|         146|   null|
|           HV0005|              B02510|2021-02-02 12:25:25|2021-02-02 12:46:33|         130|         131|   null|
|           HV0003|              B02878|2021-02-01 14:54:37|2021-02-01 15:38:30|

In [94]:
df.registerTempTable('df_feb')

/home/thomas/spark_trening/env/lib/python3.8/site-packages/pyspark/sql/dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(


In [98]:
df_result = spark.sql("""
SELECT
    dispatching_base_num,
    COUNT(1) AS num_of_trips
FROM df_feb
GROUP BY dispatching_base_num
ORDER BY num_of_trips DESC
LIMIT 1
""")

In [99]:
df_result.show()

+--------------------+------------+
|dispatching_base_num|num_of_trips|
+--------------------+------------+
|              B02510|     3233664|
+--------------------+------------+



In [100]:
!ls

 04_pyspark.ipynb	      head.csv
 env			      spark-warehouse
 fhvhv			      taxi+_zone_lookup.csv
 fhvhv_tripdata_2021-01.csv   zones
 fhvhv_tripdata_2021-02.csv  'Без названия.ipynb'
 first_connect.ipynb


In [101]:
df_zones = spark.read \
    .option("header", "true") \
    .csv('taxi+_zone_lookup.csv')

In [102]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [116]:
puzones = df.join(df_zones, (df.PULocationID == df_zones.LocationID), how='inner')

In [117]:
dozones = df.join(df_zones, (df.DOLocationID == df_zones.LocationID), how='inner')

In [119]:
puzones.registerTempTable('puzones_table')

/home/thomas/spark_trening/env/lib/python3.8/site-packages/pyspark/sql/dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(


In [122]:
df_result_2 = spark.sql("""
SELECT
    Zone,
    COUNT(1) AS number_of_count
FROM puzones_table
GROUP BY Zone
ORDER BY number_of_count DESC
""")

In [123]:
df_result_2.show()

+--------------------+---------------+
|                Zone|number_of_count|
+--------------------+---------------+
| Crown Heights North|         203777|
|       East New York|         166959|
|      Bushwick South|         140636|
|        East Village|         137901|
|Central Harlem North|         137246|
|             Bedford|         125394|
|  Stuyvesant Heights|         120026|
|            Canarsie|         117751|
|Prospect-Lefferts...|         117734|
|Flatbush/Ditmas Park|         117092|
|Washington Height...|         115590|
|             Astoria|         110787|
|TriBeCa/Civic Center|         109074|
|   East Harlem North|         103440|
|          Park Slope|         103267|
|         Brownsville|         103059|
|            Union Sq|          96469|
|         Murray Hill|          94307|
|        West Village|          93355|
|   East Harlem South|          91517|
+--------------------+---------------+
only showing top 20 rows



In [124]:
dozones.registerTempTable('dozones_table')

In [125]:
df_result_2 = spark.sql("""
SELECT
    Zone,
    COUNT(1) AS number_of_count
FROM dozones_table
GROUP BY Zone
ORDER BY number_of_count DESC
""")

In [126]:
df_result_2.show()

+--------------------+---------------+
|                Zone|number_of_count|
+--------------------+---------------+
|                  NA|         357369|
| Crown Heights North|         202250|
|       East New York|         168094|
|      Bushwick South|         140826|
|Central Harlem North|         127432|
|             Bedford|         121202|
|        East Village|         120423|
|  Stuyvesant Heights|         118566|
|Prospect-Lefferts...|         117480|
|Washington Height...|         115897|
|Flatbush/Ditmas Park|         115025|
|            Canarsie|         112124|
|             Astoria|         105745|
|         JFK Airport|         104386|
|          Park Slope|         102323|
|TriBeCa/Civic Center|          98220|
|   East Harlem North|          97698|
|         Brownsville|          97433|
|            Union Sq|          95025|
|         Murray Hill|          94618|
+--------------------+---------------+
only showing top 20 rows

